In [1]:
pip install tensorflow keras scikit-learn pandas pillow

In [4]:
# Import all necessary libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout
from keras.layers import BatchNormalization, Activation
from keras.optimizers import Adam
from keras.preprocessing.image import load_img, img_to_array
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import backend as K
import tensorflow as tf
from PIL import Image
import cv2

# Set parameters
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNELS = 1
BATCH_SIZE = 32
EPOCHS = 100

# Set data paths
BASE_PATH = '/content/drive/MyDrive/obj/salt'
TRAIN_PATH = os.path.join(BASE_PATH, 'train')
TEST_PATH = os.path.join(BASE_PATH, 'test')
DEPTHS_CSV = os.path.join(BASE_PATH, 'depths.csv')
TRAIN_CSV = os.path.join(BASE_PATH, 'train.csv')
SAMPLE_SUBMISSION_CSV = os.path.join(BASE_PATH, 'sample_submission.csv')

# Create directories if they don't exist
os.makedirs(os.path.join(TRAIN_PATH, 'images'), exist_ok=True)
os.makedirs(os.path.join(TRAIN_PATH, 'masks'), exist_ok=True)
os.makedirs(os.path.join(TEST_PATH, 'images'), exist_ok=True)

# RLE encoding/decoding functions
def rle_decode(mask_rle, shape=(101, 101)):
    """
    Decode run-length encoded mask
    """
    if mask_rle == '' or pd.isna(mask_rle):
        return np.zeros(shape)

    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

def rle_encode(img):
    """
    Encode image to run-length format
    """
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

# Dice coefficient metric (commonly used in segmentation tasks)
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

# Data preparation and augmentation
def load_and_preprocess_data():
    # Load train data
    train_df = pd.read_csv(TRAIN_CSV)
    depths_df = pd.read_csv(DEPTHS_CSV)

    # Merge with depths information
    train_df = train_df.merge(depths_df, on='id')

    images = []
    masks = []

    print("Loading training data...")
    for idx, row in train_df.iterrows():
        img_path = os.path.join(TRAIN_PATH, 'images', row['id'] + '.png')

        # Load and preprocess image
        img = load_img(img_path, color_mode='grayscale', target_size=(IMG_HEIGHT, IMG_WIDTH))
        img = img_to_array(img) / 255.0
        images.append(img)

        # Decode RLE mask
        mask = rle_decode(row['rle_mask'])
        mask = Image.fromarray(mask).resize((IMG_HEIGHT, IMG_WIDTH))
        mask = img_to_array(mask) / 255.0
        masks.append(mask)

    images = np.array(images)
    masks = np.array(masks)

    return images, masks, train_df

def load_test_data():
    # Load test data
    test_df = pd.read_csv(SAMPLE_SUBMISSION_CSV)
    test_images = []
    test_ids = []

    print("Loading test data...")
    for idx, row in test_df.iterrows():
        img_path = os.path.join(TEST_PATH, 'images', row['id'] + '.png')

        # Load and preprocess image
        img = load_img(img_path, color_mode='grayscale', target_size=(IMG_HEIGHT, IMG_WIDTH))
        img = img_to_array(img) / 255.0
        test_images.append(img)
        test_ids.append(row['id'])

    test_images = np.array(test_images)
    return test_images, test_ids

# Build U-Net model (based on zhixuhao implementation with improvements)
def unet_model():
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

    # Contracting Path
    c1 = Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    c1 = Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c1)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(0.1)(p1)

    c2 = Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    c2 = Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c2)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(0.1)(p2)

    c3 = Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    c3 = Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c3)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    p3 = MaxPooling2D((2, 2))(c3)
    p3 = Dropout(0.2)(p3)

    c4 = Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    c4 = Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c4)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    p4 = MaxPooling2D((2, 2))(c4)
    p4 = Dropout(0.2)(p4)

    # Bottleneck
    c5 = Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)
    c5 = Conv2D(1024, (3, 3), kernel_initializer='he_normal', padding='same')(c5)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)
    c5 = Dropout(0.3)(c5)

    # Expansive Path
    u6 = UpSampling2D((2, 2))(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(0.2)(u6)
    c6 = Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)
    c6 = Conv2D(512, (3, 3), kernel_initializer='he_normal', padding='same')(c6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)

    u7 = UpSampling2D((2, 2))(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(0.2)(u7)
    c7 = Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('relu')(c7)
    c7 = Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same')(c7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('relu')(c7)

    u8 = UpSampling2D((2, 2))(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(0.1)(u8)
    c8 = Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('relu')(c8)
    c8 = Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same')(c8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('relu')(c8)

    u9 = UpSampling2D((2, 2))(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(0.1)(u9)
    c9 = Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Activation('relu')(c9)
    c9 = Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same')(c9)
    c9 = BatchNormalization()(c9)
    c9 = Activation('relu')(c9)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=Adam(learning_rate=1e-4),
                  loss=dice_loss,
                  metrics=['accuracy', dice_coef])
    return model

# Data generator for training with augmentation
def data_generator(images, masks, batch_size=16, augment=True):
    num_samples = len(images)
    while True:
        indices = np.random.permutation(num_samples)
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_images = images[batch_indices]
            batch_masks = masks[batch_indices]

            if augment:
                # Apply simple augmentation
                for j in range(len(batch_images)):
                    # Random flip
                    if np.random.rand() > 0.5:
                        batch_images[j] = np.fliplr(batch_images[j])
                        batch_masks[j] = np.fliplr(batch_masks[j])

                    # Random brightness adjustment
                    if np.random.rand() > 0.5:
                        factor = 0.5 + np.random.rand()
                        batch_images[j] = np.clip(batch_images[j] * factor, 0, 1)

            yield batch_images, batch_masks

# Main execution
if __name__ == '__main__':
    # Check if GPU is available
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

    # Load and preprocess data
    X, y, train_df = load_and_preprocess_data()

    # Split data into train and validation sets
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    print(f"Training data shape: {X_train.shape}")
    print(f"Validation data shape: {X_val.shape}")

    # Build model
    model = unet_model()
    model.summary()

    # Define callbacks
    callbacks = [
        EarlyStopping(patience=15, verbose=1, restore_best_weights=True),
        ReduceLROnPlateau(factor=0.2, patience=10, min_lr=1e-7, verbose=1),
        ModelCheckpoint('salt_model_best.h5', verbose=1, save_best_only=True)
    ]

    # Train model
    print("Training model...")
    history = model.fit(
        data_generator(X_train, y_train, batch_size=BATCH_SIZE),
        steps_per_epoch=len(X_train) // BATCH_SIZE,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        callbacks=callbacks
    )

    # Save the final model
    model.save('salt_model_final.h5')
    print("Model saved as salt_model_final.h5")

    # Plot training history
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['dice_coef'], label='Training Dice Coefficient')
    plt.plot(history.history['val_dice_coef'], label='Validation Dice Coefficient')
    plt.title('Dice Coefficient')
    plt.ylabel('Dice Coefficient')
    plt.xlabel('Epoch')
    plt.legend()

    plt.tight_layout()
    plt.savefig('training_history.png')
    plt.show()

    # Load best model for prediction
    model = load_model('salt_model_best.h5',
                      custom_objects={'dice_coef': dice_coef, 'dice_loss': dice_loss})

    # Prediction on test data
    test_images, test_ids = load_test_data()
    print(f"Test data shape: {test_images.shape}")

    predictions = model.predict(test_images, batch_size=BATCH_SIZE, verbose=1)

    # Create submission
    submission = pd.read_csv(SAMPLE_SUBMISSION_CSV)

    for i, (pred, test_id) in enumerate(zip(predictions, test_ids)):
        # Resize prediction to original size (101x101)
        pred_resized = np.squeeze(pred)
        pred_resized = cv2.resize(pred_resized, (101, 101))
        pred_binary = (pred_resized > 0.5).astype(np.uint8)

        # Encode to RLE
        rle = rle_encode(pred_binary)
        submission.loc[submission['id'] == test_id, 'rle_mask'] = rle

    submission.to_csv('submission.csv', index=False)
    print("Submission file created: submission.csv")

    # Display some predictions
    fig, axes = plt.subplots(3, 3, figsize=(12, 12))
    for i in range(3):
        # Original image
        axes[i, 0].imshow(np.squeeze(test_images[i]), cmap='gray')
        axes[i, 0].set_title('Original Image')
        axes[i, 0].axis('off')

        # Prediction
        axes[i, 1].imshow(np.squeeze(predictions[i]), cmap='jet')
        axes[i, 1].set_title('Prediction')
        axes[i, 1].axis('off')

        # Thresholded prediction
        axes[i, 2].imshow((np.squeeze(predictions[i]) > 0.5).astype(np.uint8), cmap='gray')
        axes[i, 2].set_title('Thresholded Prediction')
        axes[i, 2].axis('off')

    plt.tight_layout()
    plt.savefig('predictions.png')
    plt.show()

Num GPUs Available:  1
Loading training data...
Training data shape: (3200, 128, 128, 1)
Validation data shape: (800, 128, 128, 1)


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        640 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 128, 128,  │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │     36,928 │ activation[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 128, 128,  │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 128, 128,  │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ activation_1[0][… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 64, 64,    │          0 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │     73,856 │ dropout[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_2        │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │    147,584 │ activation_2[0][… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 64, 64,    │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_3        │ (None, 64, 64,    │          0 │ batch_normalizat… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ activation_3[0][… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 32, 32,    │          0 │ max_pooling2d_1[

 Total params: 31,401,345 (119.79 MB)

 Trainable params: 31,389,569 (119.74 MB)

 Non-trainable params: 11,776 (46.00 KB)

Training model...
Epoch 1/100


AttributeError: module 'keras.backend' has no attribute 'flatten'

In [8]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Dropout, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# ------------------------------
# Parameters and paths
# ------------------------------
IMG_HEIGHT = 128
IMG_WIDTH = 128
IMG_CHANNELS = 1
BATCH_SIZE = 32
EPOCHS = 5  # adjust to 100 if needed

BASE_PATH = '/content/drive/MyDrive/obj/salt'
TRAIN_PATH = os.path.join(BASE_PATH, 'train')
TRAIN_CSV = os.path.join(BASE_PATH, 'train.csv')
DEPTHS_CSV = os.path.join(BASE_PATH, 'depths.csv')

# ------------------------------
# RLE decode
# ------------------------------
def rle_decode(mask_rle, shape=(101,101)):
    if mask_rle == '' or pd.isna(mask_rle):
        return np.zeros(shape)
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

# ------------------------------
# Load and preprocess data
# ------------------------------
def load_and_preprocess_data():
    train_df = pd.read_csv(TRAIN_CSV)
    depths_df = pd.read_csv(DEPTHS_CSV)
    train_df = train_df.merge(depths_df, on='id')

    images = []
    masks = []

    for idx, row in train_df.iterrows():
        img_path = os.path.join(TRAIN_PATH, 'images', row['id'] + '.png')
        img = load_img(img_path, color_mode='grayscale', target_size=(IMG_HEIGHT, IMG_WIDTH))
        img = img_to_array(img) / 255.0
        images.append(img)

        mask = rle_decode(row['rle_mask'])
        mask = Image.fromarray(mask).resize((IMG_HEIGHT, IMG_WIDTH))
        mask = img_to_array(mask) / 255.0
        masks.append(mask)

    return np.array(images), np.array(masks), train_df

# ------------------------------
# Data generator with augmentation
# ------------------------------
def data_generator(images, masks, batch_size=16, augment=True):
    num_samples = len(images)
    while True:
        indices = np.random.permutation(num_samples)
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_images = images[batch_indices]
            batch_masks = masks[batch_indices]

            if augment:
                for j in range(len(batch_images)):
                    if np.random.rand() > 0.5:
                        batch_images[j] = np.fliplr(batch_images[j])
                        batch_masks[j] = np.fliplr(batch_masks[j])
                    if np.random.rand() > 0.5:
                        factor = 0.5 + np.random.rand()
                        batch_images[j] = np.clip(batch_images[j] * factor, 0, 1)
            yield batch_images, batch_masks

# ------------------------------
# Dice metric
# ------------------------------
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = tf.reshape(y_true, [-1])
    y_pred_f = tf.reshape(y_pred, [-1])
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

# ------------------------------
# U-Net model
# ------------------------------
def unet_model():
    inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))

    # Contracting path
    c1 = Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal')(inputs)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    c1 = Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal')(c1)
    c1 = BatchNormalization()(c1)
    c1 = Activation('relu')(c1)
    p1 = MaxPooling2D((2,2))(c1)
    p1 = Dropout(0.1)(p1)

    c2 = Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal')(p1)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    c2 = Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal')(c2)
    c2 = BatchNormalization()(c2)
    c2 = Activation('relu')(c2)
    p2 = MaxPooling2D((2,2))(c2)
    p2 = Dropout(0.1)(p2)

    c3 = Conv2D(256, (3,3), padding='same', kernel_initializer='he_normal')(p2)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    c3 = Conv2D(256, (3,3), padding='same', kernel_initializer='he_normal')(c3)
    c3 = BatchNormalization()(c3)
    c3 = Activation('relu')(c3)
    p3 = MaxPooling2D((2,2))(c3)
    p3 = Dropout(0.2)(p3)

    c4 = Conv2D(512, (3,3), padding='same', kernel_initializer='he_normal')(p3)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    c4 = Conv2D(512, (3,3), padding='same', kernel_initializer='he_normal')(c4)
    c4 = BatchNormalization()(c4)
    c4 = Activation('relu')(c4)
    p4 = MaxPooling2D((2,2))(c4)
    p4 = Dropout(0.2)(p4)

    # Bottleneck
    c5 = Conv2D(1024, (3,3), padding='same', kernel_initializer='he_normal')(p4)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)
    c5 = Conv2D(1024, (3,3), padding='same', kernel_initializer='he_normal')(c5)
    c5 = BatchNormalization()(c5)
    c5 = Activation('relu')(c5)
    c5 = Dropout(0.3)(c5)

    # Expansive path
    u6 = UpSampling2D((2,2))(c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(0.2)(u6)
    c6 = Conv2D(512, (3,3), padding='same', kernel_initializer='he_normal')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)
    c6 = Conv2D(512, (3,3), padding='same', kernel_initializer='he_normal')(c6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('relu')(c6)

    u7 = UpSampling2D((2,2))(c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(0.2)(u7)
    c7 = Conv2D(256, (3,3), padding='same', kernel_initializer='he_normal')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('relu')(c7)
    c7 = Conv2D(256, (3,3), padding='same', kernel_initializer='he_normal')(c7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('relu')(c7)

    u8 = UpSampling2D((2,2))(c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(0.1)(u8)
    c8 = Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('relu')(c8)
    c8 = Conv2D(128, (3,3), padding='same', kernel_initializer='he_normal')(c8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('relu')(c8)

    u9 = UpSampling2D((2,2))(c8)
    u9 = concatenate([u9, c1])
    u9 = Dropout(0.1)(u9)
    c9 = Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Activation('relu')(c9)
    c9 = Conv2D(64, (3,3), padding='same', kernel_initializer='he_normal')(c9)
    c9 = BatchNormalization()(c9)
    c9 = Activation('relu')(c9)

    outputs = Conv2D(1, (1,1), activation='sigmoid')(c9)

    model = Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=Adam(1e-4), loss=dice_loss, metrics=['accuracy', dice_coef])
    return model

# ------------------------------
# Callbacks
# ------------------------------
callbacks = [
    EarlyStopping(patience=15, verbose=1, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.2, patience=10, min_lr=1e-7, verbose=1),
    ModelCheckpoint('salt_model_best.h5', verbose=1, save_best_only=True)
]

# ------------------------------
# Load data
# ------------------------------
X, y, train_df = load_and_preprocess_data()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# ------------------------------
# Train model
# ------------------------------
model = unet_model()
history = model.fit(
    data_generator(X_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X_train)//BATCH_SIZE,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    callbacks=callbacks
)


Epoch 1/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 473ms/step - accuracy: 0.5139 - dice_coef: 0.0032 - loss: 0.9968
Epoch 1: val_loss improved from inf to 0.99797, saving model to salt_model_best.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 145s 658ms/step - accuracy: 0.5151 - dice_coef: 0.0032 - loss: 0.9968 - val_accuracy: 0.0025 - val_dice_coef: 0.0020 - val_loss: 0.9980 - learning_rate: 1.0000e-04
Epoch 2/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step - accuracy: 0.7301 - dice_coef: 0.0052 - loss: 0.9948
Epoch 2: val_loss improved from 0.99797 to 0.99769, saving model to salt_model_best.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 57s 574ms/step - accuracy: 0.7301 - dice_coef: 0.0052 - loss: 0.9948 - val_accuracy: 0.1505 - val_dice_coef: 0.0023 - val_loss: 0.9977 - learning_rate: 1.0000e-04
Epoch 3/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step - accuracy: 0.7400 - dice_coef: 0.0059 - loss: 0.9941
Epoch 3: val_loss improved from 0.99769 to 0.99662, saving model to salt_model_best.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 57s 574ms/step - accuracy: 0.7400 - dice_coef: 0.0059 - loss: 0.9941 - val_accuracy: 0.4676 - val_dice_coef: 0.0034 - val_loss: 0.9966 - learning_rate: 1.0000e-04
Epoch 4/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step - accuracy: 0.7314 - dice_coef: 0.0063 - loss: 0.9937
Epoch 4: val_loss improved from 0.99662 to 0.99564, saving model to salt_model_best.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 55s 553ms/step - accuracy: 0.7315 - dice_coef: 0.0063 - loss: 0.9937 - val_accuracy: 0.5852 - val_dice_coef: 0.0044 - val_loss: 0.9956 - learning_rate: 1.0000e-04
Epoch 5/5
100/100 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step - accuracy: 0.7483 - dice_coef: 0.0065 - loss: 0.9935
Epoch 5: val_loss improved from 0.99564 to 0.99413, saving model to salt_model_best.h5


100/100 ━━━━━━━━━━━━━━━━━━━━ 58s 576ms/step - accuracy: 0.7483 - dice_coef: 0.0065 - loss: 0.9935 - val_accuracy: 0.6912 - val_dice_coef: 0.0059 - val_loss: 0.9941 - learning_rate: 1.0000e-04
Restoring model weights from the end of the best epoch: 5.
